In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
# What all do we have? We have the geenrated features, and the densities defined for all the test blocks.

In [3]:
# Do the data prep for one block, and define a function for the rest

In [5]:
# densities are here
block_0103_density_file = np.load(os.path.join("data/densities_overlapping/", "all_densities_block_0103.npy"))

In [6]:
block_0103_density_file.shape

(3072, 20, 1)

In [7]:
# Where are the feature files?

In [8]:
# for test time points
block_0103_feature_file_test_points = np.load(os.path.join("data/predicted_sequences_from_stage_1", "pred_values_blk_0103.npy"))

In [9]:
block_0103_feature_file_test_points.shape

(3072, 7, 32)

In [10]:
# for train time points
block_0103_feature_file_train_points = np.load(os.path.join("data/train_features_non_overlapping/", "train_features_block_0103.npy"))

In [11]:
block_0103_feature_file_train_points.shape

(3072, 13, 32)

In [12]:
# Stack the featrue files together?

In [13]:
block_0103_features = np.hstack((block_0103_feature_file_train_points, block_0103_feature_file_test_points))

In [14]:
block_0103_features.shape

(3072, 20, 32)

In [15]:
# alternative way
block_0103_features_alt = np.concatenate((block_0103_feature_file_train_points, block_0103_feature_file_test_points), axis = 1)

In [16]:
block_0103_features_alt.shape

(3072, 20, 32)

In [17]:
# sanity check
np.mean(block_0103_features == block_0103_features_alt)

1.0

In [18]:
# now stack this with the response?

In [19]:
block_0103_all_data = np.concatenate((block_0103_features, block_0103_density_file), axis = 2)

In [20]:
block_0103_all_data.shape

(3072, 20, 33)

In [21]:
block_0103_all_data_alt = np.dstack((block_0103_features, block_0103_density_file))

In [22]:
block_0103_all_data_alt.shape

(3072, 20, 33)

In [23]:
# sanity check
np.mean(block_0103_all_data_alt == block_0103_all_data)

1.0

In [24]:
# Okay, now need to create dfs, and store so these can be later used for the BLAR model

In [25]:
block_0103_all_data_df_100 = pd.DataFrame(block_0103_all_data[100,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density'])

In [27]:
%%time
counter = 0
for row in range(block_0103_all_data.shape[0]):
    # create the df
    current_df = pd.DataFrame(block_0103_all_data[row,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density'])
    # name the df
    df_name = 'block_0103_df_' + str(counter) + '.csv'
    # df path
    df_path = "data/BLAR_ready_dfs/block_0103"
    # save the df
    current_df.to_csv(os.path.join(df_path, df_name), index = False)
    counter = counter + 1

CPU times: user 2.82 s, sys: 143 ms, total: 2.96 s
Wall time: 5.58 s


In [29]:
# block_0103_all_data_df_100

In [30]:
# Seems this is what we need, but do sanity checks

In [31]:
# Okay, how do we ensure we have done this correct

In [32]:
block_0103_features.shape

(3072, 20, 32)

In [33]:
# sanity check for features
# train tps
print(np.mean(block_0103_features[:,:13,:] == block_0103_feature_file_train_points))
# test tps
print(np.mean(block_0103_features[:,13:,:] == block_0103_feature_file_test_points))

1.0
1.0


In [34]:
# Also check a few random dfs
block_0103_all_data_df_100 = pd.DataFrame(block_0103_all_data[100,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density']) # matches with the stored df
block_0103_all_data_df_500 = pd.DataFrame(block_0103_all_data[500,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density']) # matches with the stored df
block_0103_all_data_df_785 = pd.DataFrame(block_0103_all_data[785,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density']) # matches with the stored df
block_0103_all_data_df_91 = pd.DataFrame(block_0103_all_data[91,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density']) # matches with the stored df

In [36]:
# block_0103_all_data_df_91

In [37]:
# Okay, define a function to be used for the rest of the blocks

In [38]:
# What all do we need for the function?

# density_file_loc
# feature_file_loc_train
# feature_file_loc_test


In [39]:
def store_BLAR_ready_dfs(density_file_loc, feature_file_loc_train, feature_file_loc_test, block_name_for_df, save_df_path):
    # load density file
    density_array = np.load(density_file_loc)
    # load train features
    train_features = np.load(feature_file_loc_train)
    # load test featrues
    test_features = np.load(feature_file_loc_test)
    # stack features files
    stacked_features = np.hstack((train_features, test_features))
    # do sanity check
    stacked_features_alt = np.concatenate((train_features, test_features), axis = 1)
    print(np.mean(stacked_features == stacked_features_alt))
    # print shape
    print(stacked_features.shape)
    # stack features and density
    all_data_array = np.concatenate((stacked_features, density_array), axis = 2)
    # let's also do teh sanity checks
    all_data_alt = np.dstack((stacked_features, density_array))
    print(np.mean(all_data_array == all_data_alt))
    print(all_data_array.shape)
    # do the for loop and store the dfs

    catch_dfs = []
    counter = 0
    for row in range(all_data_array.shape[0]):
        # create the df
        current_df = pd.DataFrame(all_data_array[row,:,:], columns = ['feature_' + str(i) for i in range(32)] + ['density'])
        catch_dfs.append(current_df)
        # name the df
        df_name = block_name_for_df + str(counter) + '.csv'
        # save the df
        current_df.to_csv(os.path.join(save_df_path, df_name), index = False)
        counter = counter + 1

    return catch_dfs

In [40]:
# Try this for block 0103 - we need to create an additional folder for the block names

In [43]:
# argument values
dense_file_loc = "data/densities_overlapping/all_densities_block_0103.npy"
train_features_loc = "data/train_features_non_overlapping/train_features_block_0103.npy"
test_features_loc = "data/predicted_sequences_from_stage_1/pred_values_blk_0103.npy"
blk_name = "block_0103_df_"
save_df_loc = "data/BLAR_ready_dfs/block_0103"

In [44]:
try_0103 = store_BLAR_ready_dfs(dense_file_loc, train_features_loc, test_features_loc, blk_name, save_df_loc)

1.0
(3072, 20, 32)
1.0
(3072, 20, 33)


In [45]:
# are the all the same?

# list files in "data/BLAR_ready_dfs/model_1" for block 0103

In [46]:
all_files_here = [file for file in os.listdir("data/BLAR_ready_dfs")]

In [47]:
len(all_files_here)

14

In [48]:
only_csv  = [file for file in os.listdir("data/BLAR_ready_dfs/block_0103") if file.split(".")[-1] == 'csv']

In [49]:
# only_csv

In [50]:
len(only_csv)

3072

In [51]:
# files inside the block_0103 folder
only_csv_block_0103 = [file for file in os.listdir(save_df_loc)]

In [52]:
len(only_csv_block_0103)

3072

In [53]:
csv_files = ['block_0103_df_' + str(i) + '.csv' for i in range(3072)]

In [55]:
# csv_files

In [56]:
%%time
catch_true = []
for file in csv_files:
    load_from_loc1 = pd.read_csv(os.path.join("data/BLAR_ready_dfs/block_0103", file))
    load_from_loc2 = pd.read_csv(os.path.join(save_df_loc, file))
    is_true = load_from_loc1.equals(load_from_loc2)
    catch_true.append(is_true)

CPU times: user 7.08 s, sys: 148 ms, total: 7.23 s
Wall time: 12.8 s


In [57]:
np.mean(catch_true)

1.0

In [58]:
# Let's preprocess data for teh rest of the blocks

In [59]:
# block 0104

In [62]:
# argument values
dense_file_loc_0104 = "data/densities_overlapping/all_densities_block_0104.npy"
train_features_loc_0104 = "data/train_features_non_overlapping/train_features_block_0104.npy"
test_features_loc_0104 = "data/predicted_sequences_from_stage_1/pred_values_blk_0104.npy"
blk_name_0104 = "block_0104_df_"
save_df_loc_0104 = "data/BLAR_ready_dfs/block_0104"

In [63]:
try_0104 = store_BLAR_ready_dfs(dense_file_loc_0104, train_features_loc_0104, test_features_loc_0104, blk_name_0104, save_df_loc_0104)

1.0
(3072, 20, 32)
1.0
(3072, 20, 33)


In [65]:
# block 0105

In [66]:
# argument values
dense_file_loc_0105 = "data/densities_overlapping/all_densities_block_0105.npy"
train_features_loc_0105 = "data/train_features_non_overlapping/train_features_block_0105.npy"
test_features_loc_0105 = "data/predicted_sequences_from_stage_1/pred_values_blk_0105.npy"
blk_name_0105 = "block_0105_df_"
save_df_loc_0105 = "data/BLAR_ready_dfs/block_0105"

In [67]:
try_0105 = store_BLAR_ready_dfs(dense_file_loc_0105, train_features_loc_0105, test_features_loc_0105, blk_name_0105, save_df_loc_0105)

1.0
(3072, 20, 32)
1.0
(3072, 20, 33)


In [68]:
# block 0106

In [69]:
# argument values
dense_file_loc_0106 = "data/densities_overlapping/all_densities_block_0106.npy"
train_features_loc_0106 = "data/train_features_non_overlapping/train_features_block_0106.npy"
test_features_loc_0106 = "data/predicted_sequences_from_stage_1/pred_values_blk_0106.npy"
blk_name_0106 = "block_0106_df_"
save_df_loc_0106 = "data/BLAR_ready_dfs/block_0106"

In [70]:
try_0106 = store_BLAR_ready_dfs(dense_file_loc_0106, train_features_loc_0106, test_features_loc_0106, blk_name_0106, save_df_loc_0106)

1.0
(3072, 20, 32)
1.0
(3072, 20, 33)


In [71]:
# block 0201

In [72]:
# argument values
dense_file_loc_0201 = "data/densities_overlapping/all_densities_block_0201.npy"
train_features_loc_0201 = "data/train_features_non_overlapping/train_features_block_0201.npy"
test_features_loc_0201 = "data/predicted_sequences_from_stage_1/pred_values_blk_0201.npy"
blk_name_0201 = "block_0201_df_"
save_df_loc_0201 = "data/BLAR_ready_dfs/block_0201"

In [73]:
try_0201 = store_BLAR_ready_dfs(dense_file_loc_0201, train_features_loc_0201, test_features_loc_0201, blk_name_0201, save_df_loc_0201)

1.0
(3072, 20, 32)
1.0
(3072, 20, 33)


In [74]:
# block 0202

In [75]:
# argument values
dense_file_loc_0202 = "data/densities_overlapping/all_densities_block_0202.npy"
train_features_loc_0202 = "data/train_features_non_overlapping/train_features_block_0202.npy"
test_features_loc_0202 = "data/predicted_sequences_from_stage_1/pred_values_blk_0202.npy"
blk_name_0202 = "block_0202_df_"
save_df_loc_0202 = "data/BLAR_ready_dfs/block_0202"

In [76]:
try_0202 = store_BLAR_ready_dfs(dense_file_loc_0202, train_features_loc_0202, test_features_loc_0202, blk_name_0202, save_df_loc_0202)

1.0
(3072, 20, 32)
1.0
(3072, 20, 33)


In [77]:
# block 0205

In [78]:
# argument values
dense_file_loc_0205 = "data/densities_overlapping/all_densities_block_0205.npy"
train_features_loc_0205 = "data/train_features_non_overlapping/train_features_block_0205.npy"
test_features_loc_0205 = "data/predicted_sequences_from_stage_1/pred_values_blk_0205.npy"
blk_name_0205 = "block_0205_df_"
save_df_loc_0205 = "data/BLAR_ready_dfs/block_0205"

In [79]:
try_0205 = store_BLAR_ready_dfs(dense_file_loc_0205, train_features_loc_0205, test_features_loc_0205, blk_name_0205, save_df_loc_0205)

1.0
(3072, 20, 32)
1.0
(3072, 20, 33)


In [80]:
# block 0206

In [81]:
# argument values
dense_file_loc_0206 = "data/densities_overlapping/all_densities_block_0206.npy"
train_features_loc_0206 = "data/train_features_non_overlapping/train_features_block_0206.npy"
test_features_loc_0206 = "data/predicted_sequences_from_stage_1/pred_values_blk_0206.npy"
blk_name_0206 = "block_0206_df_"
save_df_loc_0206 = "data/BLAR_ready_dfs/block_0206"

In [82]:
try_0206 = store_BLAR_ready_dfs(dense_file_loc_0206, train_features_loc_0206, test_features_loc_0206, blk_name_0206, save_df_loc_0206)

1.0
(3072, 20, 32)
1.0
(3072, 20, 33)


In [83]:
# block 0302

In [84]:
# argument values
dense_file_loc_0302 = "data/densities_overlapping/all_densities_block_0302.npy"
train_features_loc_0302 = "data/train_features_non_overlapping/train_features_block_0302.npy"
test_features_loc_0302 = "data/predicted_sequences_from_stage_1/pred_values_blk_0302.npy"
blk_name_0302 = "block_0302_df_"
save_df_loc_0302 = "data/BLAR_ready_dfs/block_0302"

In [85]:
try_0302 = store_BLAR_ready_dfs(dense_file_loc_0302, train_features_loc_0302, test_features_loc_0302, blk_name_0302, save_df_loc_0302)

1.0
(3072, 20, 32)
1.0
(3072, 20, 33)


In [86]:
# block 0303

In [87]:
# argument values
dense_file_loc_0303 = "data/densities_overlapping/all_densities_block_0303.npy"
train_features_loc_0303 = "data/train_features_non_overlapping/train_features_block_0303.npy"
test_features_loc_0303 = "data/predicted_sequences_from_stage_1/pred_values_blk_0303.npy"
blk_name_0303 = "block_0303_df_"
save_df_loc_0303 = "data/BLAR_ready_dfs/block_0303"

In [88]:
try_0303 = store_BLAR_ready_dfs(dense_file_loc_0303, train_features_loc_0303, test_features_loc_0303, blk_name_0303, save_df_loc_0303)

1.0
(3072, 20, 32)
1.0
(3072, 20, 33)


In [89]:
# block 0304

In [90]:
# argument values
dense_file_loc_0304 = "data/densities_overlapping/all_densities_block_0304.npy"
train_features_loc_0304 = "data/train_features_non_overlapping/train_features_block_0304.npy"
test_features_loc_0304 = "data/predicted_sequences_from_stage_1/pred_values_blk_0304.npy"
blk_name_0304 = "block_0304_df_"
save_df_loc_0304 = "data/BLAR_ready_dfs/block_0304"

In [91]:
try_0304 = store_BLAR_ready_dfs(dense_file_loc_0304, train_features_loc_0304, test_features_loc_0304, blk_name_0304, save_df_loc_0304)

1.0
(3072, 20, 32)
1.0
(3072, 20, 33)


In [92]:
# block 0305

In [95]:
# argument values
dense_file_loc_0305 = "data/densities_overlapping/all_densities_block_0305.npy"
train_features_loc_0305 = "data/train_features_non_overlapping/train_features_block_0305.npy"
test_features_loc_0305 = "data/predicted_sequences_from_stage_1/pred_values_blk_0305.npy"
blk_name_0305 = "block_0305_df_"
save_df_loc_0305 = "data/BLAR_ready_dfs/block_0305"

In [96]:
try_0305 = store_BLAR_ready_dfs(dense_file_loc_0305, train_features_loc_0305, test_features_loc_0305, blk_name_0305, save_df_loc_0305)

1.0
(3072, 20, 32)
1.0
(3072, 20, 33)


In [97]:
# block 0306

In [98]:
# argument values
dense_file_loc_0306 = "data/densities_overlapping/all_densities_block_0306.npy"
train_features_loc_0306 = "data/train_features_non_overlapping/train_features_block_0306.npy"
test_features_loc_0306 = "data/predicted_sequences_from_stage_1/pred_values_blk_0306.npy"
blk_name_0306 = "block_0306_df_"
save_df_loc_0306 = "data/BLAR_ready_dfs/block_0306"

In [99]:
try_0306 = store_BLAR_ready_dfs(dense_file_loc_0306, train_features_loc_0306, test_features_loc_0306, blk_name_0306, save_df_loc_0306)

1.0
(3072, 20, 32)
1.0
(3072, 20, 33)
